In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import sqlalchemy as sa
from dotenv import load_dotenv
import os

In [ ]:
# We have the countries.json -> link -> dataframe using pandas.
# Iterate over the countries dataframe -> legislative_csv_urls
# Moving to a database.-> to_sql method to move it to a database.

In [2]:
df = pd.read_json("https://cdn.rawgit.com/everypolitician/everypolitician-data/080cb46/countries.json")

In [24]:
df.head()

,name,country,code,slug,legislatures
0,Abkhazia,Abkhazia,GE-AB,Abkhazia,"[{'name': 'People's Assembly', 'slug': 'Assemb..."
1,Afghanistan,Afghanistan,AF,Afghanistan,"[{'name': 'Wolesi Jirga', 'slug': 'Wolesi-Jirg..."
2,Albania,Albania,AL,Albania,"[{'name': 'Kuvendi', 'slug': 'Assembly', 'sour..."
3,Alderney,Alderney,GG-ALD,Alderney,"[{'name': 'States', 'slug': 'States', 'sources..."
4,Algeria,Algeria,DZ,Algeria,"[{'name': 'People's National Assembly', 'slug'..."


In [25]:
def get_csv_urls(df):
    csv_dict = {}
    for idx, row in df.iterrows():
        country_name = row["country"]
        country_code = row["code"]
        for leg in row["legislatures"]:
            leg_name = leg["name"]
            for leg_period in leg["legislative_periods"]:
                st_dt = leg_period["start_date"]
                if "end_date" in leg_period.keys():
                    ed_dt = leg_period["end_date"]
                else:
                    ed_dt = ""
                csv_url = leg_period["csv_url"]
                csv_dict[(country_name, country_code, leg_name, st_dt, ed_dt)] = csv_url
    return csv_dict

In [26]:
urls = get_csv_urls(df)

In [36]:
def get_politician_data(pol_urls):
    all_politician_df = []
    for keys, value in tqdm(pol_urls.items()):
        try:
            politician_df = pd.read_csv(value)
            politician_df["country_name"] = keys[0]
            politician_df["country_code"] = keys[1]
            politician_df["leg_name"] = keys[2]
            politician_df["start_date_legislature"] = keys[3]
            politician_df["end_date_legislature"] = keys[4]
            all_politician_df.append(politician_df)
        except Exception as e:
            print(f"For the url {value} got error {e}")
    return pd.concat(all_politician_df)
        
        

In [38]:
pol_df = get_politician_data(urls)

HBox(children=(FloatProgress(value=0.0, max=672.0), HTML(value='')))

For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/0b326e2/data/Ecuador/Asamblea/term-2013.csv got error HTTP Error 404: Not Found
For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/8102167/data/Montserrat/Assembly/term-1.csv got error HTTP Error 404: Not Found
For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/fbd3ca5/data/Norway/Storting/term-1969-73.csv got error HTTP Error 404: Not Found
For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/fbd3ca5/data/Norway/Storting/term-1965-69.csv got error HTTP Error 404: Not Found
For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/6baa026/data/Seychelles/Assembly/term-2011.csv got error HTTP Error 404: Not Found
For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/fedacb2/data/Turkmenistan/Mejlis/term-5.csv got error HTTP Error 404: Not Found
For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/fedacb2/d

In [20]:
usa = pol_df[pol_df.country_code=='US']

In [43]:
pol_df.to_csv("politician_data.csv", index=False)Abkhazia

In [8]:
pol_df = pd.read_csv("politician_data.csv", encoding='utf-8')

In [13]:
class Connection(object):
    def __init__(self):
        '''
        Load the environment variables by calling the load_dotenv method.
        The database connection details are set as environment variables.
        '''
        load_dotenv()
    
    def connect_to_database(self, use_postgres=True, return_as_string=False):
        '''
        Connect to the mysql database and create the engine object
        using sqlalchemy.
        '''
        if use_postgres:
            conn_string = f'postgres://{os.getenv("user")}:{os.getenv("password")}@{os.getenv("host")}:{os.getenv("port")}/{os.getenv("database")}?charset=utf8'
        else:
            conn_string = f'mysql://{os.getenv("user")}:{os.getenv("password")}@{os.getenv("host")}:{os.getenv("port")}/{os.getenv("database")}?charset=utf8'
        if return_as_string:
            return conn_string
        engine = sa.create_engine(conn_string)
        return engine



In [14]:
conn = Connection()
engine = conn.connect_to_database(use_postgres=False, return_as_string=True)

In [21]:
usa.to_sql("every_politician", engine, index=False, if_exists = 'append')

In [22]:
# Figure, out the unicode for entering the other characters to a database.